# A/B Testing

### data 준비 - 이산변수 비교

In [2]:
import numpy.random

In [3]:
a = numpy.random.uniform(size=50) > 0.5
a

array([False, False,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False, False,  True,  True,
       False, False, False,  True, False, False,  True,  True, False,
        True,  True,  True,  True, False,  True, False, False, False,
       False,  True,  True, False,  True,  True,  True, False, False,
        True, False,  True, False,  True], dtype=bool)

In [4]:
a.sum()

28

In [5]:
b = numpy.random.uniform(size=50) > 0.5
b

array([ True, False, False, False,  True,  True,  True, False, False,
        True, False,  True, False, False,  True,  True, False, False,
       False,  True,  True,  True, False, False, False,  True, False,
        True, False,  True, False, False, False,  True, False,  True,
        True, False,  True, False, False, False, False, False,  True,
       False, False,  True, False,  True], dtype=bool)

In [6]:
b.sum()

21

### statsmodels

In [8]:
from statsmodels.stats.proportion import proportions_chisquare

In [9]:
proportions_chisquare([a.sum(), b.sum()], [len(a), len(b)])

(1.9607843137254903, 0.16142946236707922, (array([[28, 22],
         [21, 29]]), array([[ 24.5,  25.5],
         [ 24.5,  25.5]])))

* 카이제곱 = 1.96
* p = 0.16

In [12]:
import pandas as pd

In [15]:
def calc_chisq(a, b):
    chisq = 0
    total = len(a) + len(b)
    all_pos = a.sum() + b.sum()
    prob = all_pos / total
    for x in [a, b]:
        positive = len(x) * prob
        chisq += (x.sum() - positive) ** 2 / positive

        negative = len(x) - positive
        chisq += (len(x) - x.sum() - negative)** 2 / negative
    return chisq

In [16]:
calc_chisq(a, b)

1.9607843137254903

### bootstrapping

In [7]:
# a, b 경우를 합쳐서 무작위로 다시 두 집단으로 나눠서 카이제곱 분포를 생성

In [17]:
import numpy as np

In [18]:
merge = np.hstack((a, b))
merge

array([False, False,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True, False, False,  True,  True,
       False, False, False,  True, False, False,  True,  True, False,
        True,  True,  True,  True, False,  True, False, False, False,
       False,  True,  True, False,  True,  True,  True, False, False,
        True, False,  True, False,  True,  True, False, False, False,
        True,  True,  True, False, False,  True, False,  True, False,
       False,  True,  True, False, False, False,  True,  True,  True,
       False, False, False,  True, False,  True, False,  True, False,
       False, False,  True, False,  True,  True, False,  True, False,
       False, False, False, False,  True, False, False,  True, False,  True], dtype=bool)

In [20]:
np.random.choice(merge, 50)

array([ True, False, False,  True, False, False,  True,  True, False,
        True, False, False, False, False,  True,  True,  True, False,
        True,  True,  True, False, False,  True,  True, False, False,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True,  True, False, False, False, False,  True,
       False,  True,  True, False, False], dtype=bool)

In [22]:
dist = []
for _ in range(1000):   # 테스트 1000 번 반복
    s1 = numpy.random.choice(merge, 50)
    s2 = numpy.random.choice(merge, 50)
    c = calc_chisq(s1, s2)
    dist.append(c)

In [23]:
np.sum(np.array(dist) > calc_chisq(a, b))

185

In [24]:
185 / (100 * 1000)

0.00185

## 연속 변수의 비교